In [229]:
import re
import datetime as dt
import time
import numpy as np
#import langdetect
import pandas as pd
 
def getLikes(html):    
    search_word = '<a class="counter" (.+?)</a>'
    Likes = re.compile(search_word,re.DOTALL)
    LikesList = re.findall(Likes,html)
    return LikesList

def getAuthor(html):
    search_word = '<a class="questionersName" href="(.+?)</a>'
    Author= re.compile(search_word,re.DOTALL)
    AuthorList = re.findall(Author,html)
    return AuthorList

def getAnswer(html):
    search_word = '<div class="answerWrapper">(.+?)<p class="readMore">'   
    search_word = '<div class="answerWrapper">(.+?)<div class="streamItemContent streamItemContent-footer">'
    Answer = re.compile(search_word,re.DOTALL)
    AnswerList = re.findall(Answer,html)
    return AnswerList

def getQuestion(html):
    search_word =  '<div class="streamItemContent streamItemContent-question">(.+?)</div>'
    Question = re.compile(search_word,re.DOTALL)
    QuestionList = re.findall(Question,html)
    return QuestionList

def getTimeofPost(html):
    
    search_word = '<a class="streamItemsAge" title="(.+?)" href='
    search_word = '<div class="streamItemsAge links">(.+?)" href='
    AnswerDate = re.compile(search_word,re.DOTALL)
    AnswerDateList = re.findall(AnswerDate,html)
    return AnswerDateList

def getLink(html):
    search_word = '<div class="streamItemContent streamItemContent-visual">  <div class="streamItem-visualItem"><a (.+?) /></a></div>'
    Link = re.compile(search_word,re.DOTALL)
    LinkList = re.findall(Link,html)
    return LinkList

In [8]:
f = open('../data/raw_data/fulluserdata_start34_1.txt','r')
lines = f.read()
users = lines.split("username:")

In [238]:
columns = ['question','answer','likes','date']
df = pd.DataFrame(columns=columns)
for n in range(1,len(users)-1):#
    temp = pd.DataFrame([["q","an","l","t"]],columns=columns)

    t = getTimeofPost(users[n])
    q = getQuestion(users[n])
    an = getAnswer(users[n])
    l = getLikes(users[n])
    #u = len(t)*usr
    try:
        d = {'question': q, 'answer': an, 'likes':l, 'date':t}
        temp = pd.DataFrame(data=d)
        df = df.append(temp,ignore_index=True)
    except:
        print n
        print len(q),len(an),len(t),len(l)

In [239]:
df.head()

,answer,date,likes,question
0,"\n <p class=""streamItemContent streamItemCo...","\n \n <a title=""March 28, 2017 14:23...","href=""/MzelleClem/answers/140838031710"">2",\n <h2>As-tu déjà menti sur ton âge?</h2>...
1,"\n <p class=""streamItemContent streamItemCo...","\n \n <a title=""March 27, 2017 11:12...","style=""display:none"" href=""/MzelleClem/answers...",\n <h2>À quand remonte la dernière fois q...
2,"\n <p class=""streamItemContent streamItemCo...","\n \n <a title=""March 23, 2017 07:30...","style=""display:none"" href=""/MzelleClem/answers...",\n <h2>Douche ou bain?</h2>\n \n
3,"\n <p class=""streamItemContent streamItemCo...","\n \n <a title=""March 22, 2017 12:29...","style=""display:none"" href=""/MzelleClem/answers...",\n <h2>Quel est ton niveau de jalousie su...
4,"\n <p class=""streamItemContent streamItemCo...","\n \n <a title=""March 22, 2017 12:28...","style=""display:none"" href=""/MzelleClem/answers...",\n <h2>Quelle compétence aimerais-tu appr...


In [240]:
## clean likes
f = lambda x: int(x[x.find('>')+1:])
df['likes']=df['likes'].apply(f)


## clean date
f = lambda x: x[x.find('title')+7:]
df['date'] = df['date'].apply(f)





In [241]:
## clean question
f = lambda x: str(x)
df['question'] = df['question'].apply(f)

f = lambda x: x[x.find('<h2>')+4:x.find('</h2>')]
df['question'] = df['question'].apply(f)


def getLinks(txt):
    search_word = 'http://ask.fm/(.+?)</a>'
    link = re.compile(search_word)
    links = re.findall(link,txt)
    if len(links)>0:
        return True
    else:
        return False

f = lambda x: x.replace(x[x.find('<a target'):x.find('</a>')+4],'$URL') if getLinks(x) else x
df['question'] = df['question'].apply(f)

f = lambda x: getAuthor(x)
df['author'] = df['question'].apply(f)

f = lambda x: x[0][:x[0].find('"')] if len(x)>0 else ""
df['author'] = df['author'].apply(f)

In [242]:
## get answer

    
def case1(txt): # text only
    search_word = '<p class="streamItemContent streamItemContent-answer">(.+?)</p>'
    answer = re.findall(re.compile(search_word,re.DOTALL),txt)
    return answer[0]

def case2(txt): # text and media
    search_word = '<p class="streamItemContent streamItemContent-answer">(.+?)</p>'
    answer = re.findall(re.compile(search_word,re.DOTALL),txt)
    return answer[0]+' $URL_media'
        
def case3(txt): # media only
    answer = '$URL_media'
    return answer
    
def getCleanAnswer(x):    
    if x.find('streamItemContent streamItemContent-answer">')>-1:
        if x.find('streamItemContent streamItemContent-visual')>-1:
            xx = case2(x)
        else:
            xx = case1(x)    
    else:
        xx = case3(x)
    return xx
        
f = lambda x: getCleanAnswer(x)
df['temp'] = df['answer'].apply(f)


In [243]:

## get media link in the answer

def image(txt):
    search_word = '<div class="streamItemContent streamItemContent-visual">(.+?)</div>'
    image_links = re.findall(re.compile(search_word,re.DOTALL),txt)
    image_links = image_links[0]

    image_links = image_links[image_links.find('https:'):]
    image_link = image_links[:image_links.find('"')]
        
    if len(image_link)<4:
        print 'old ',image_links
    return image_link
    
    
def case3(txt): # media only
    answer = '$URL_media'
    return answer
    
def getLinkAnswer(x):    
    if x.find('streamItemContent streamItemContent-answer')>0:
        if x.find('streamItemContent streamItemContent-visual')>0:
            xx = image(x)
        else:
            xx = ''   
    else:
        xx = image(x)
    return xx
        
f = lambda x: getLinkAnswer(x)
df['answer_links'] = df['answer'].apply(f)


df['answer'] = df['temp']

del df['temp']

In [247]:
df.head()

,answer,date,likes,question,author,answer_links
0,Non,"March 28, 2017 14:23:16 GMT",2,As-tu déjà menti sur ton âge?,,
1,Je commande pas je les fais,"March 27, 2017 11:12:13 GMT",0,À quand remonte la dernière fois que tu as com...,,
2,Les deux,"March 23, 2017 07:30:32 GMT",0,Douche ou bain?,,
3,5/6,"March 22, 2017 12:29:38 GMT",0,Quel est ton niveau de jalousie sur une échell...,,
4,La langue des signes parce que je trouve ça su...,"March 22, 2017 12:28:27 GMT",0,Quelle compétence aimerais-tu apprendre et pou...,,


In [ ]:
# this is the pattern when there is txt+image in answer

'  <div class="answerWrapper">
    <p class="streamItemContent streamItemContent-answer">Kim Seeve</p>
    <p class="readMore">
      <a class="button" data-action="ReadMore" href="#">View more</a>
    </p>
  </div>

    <div class="streamItemContent streamItemContent-visual">  <div class="streamItem-visualItem">
    <a data-gtm="{&quot;name&quot;:&quot;popup&quot;,&quot;meta&quot;:[&quot;media&quot;,&quot;Media&quot;,
    &quot;Photo&quot;]}" 
    data-action="ImageOpen" data-url="https://akphoto1.ask.fm/990/462/749/150003011-1rr4353-khgf6gt53qdrl7f/
    large/file.jpg" href="/KimSeverr/answers/68013999121/photo"><img onerror="this.src=&#39;https://
    static.ask.fm/assets/no-image-0b30a778d766680b5d174333c571a1fa4189ffded12bc17da057d8de0900f032.png&#39;;
    this.onerror=&#39;&#39;" src="https://akphoto1.ask.fm/990/462/749/150003011-1rr4353-khgf6gt53qdrl7f/original/file.jpg" /></a></div>
</div>'

  <div class="streamItemContent streamItemContent-footer">



# this is the pattern when there is only image


  <div class="answerWrapper">
    
    <p class="readMore">
      <a class="button" data-action="ReadMore" href="#">View more</a>
    </p>
  </div>

    <div class="streamItemContent streamItemContent-visual">  
    <div class="streamItem-visualItem"><a data-gtm="
    {&quot;name&quot;:&quot;popup&quot;,&quot;meta&quot;:
     [&quot;media&quot;,&quot;Media&quot;,&quot;Photo&quot;]}" 
    data-action="ImageOpen" data-url="https://akphoto2.ask.fm/a5b/55ee9/e808/4b63/8799/c04c81425948/large/3310423.jpg" 
    href="/nicoliersch16/answers/140600674898/photo"><img onerror="this.src=&#39;https://static.ask.fm/assets/no-image-0b30a778d766680b5d174333c571a1fa4189ffded12bc17da057d8de0900f032.png&#39;;this.onerror=&#39;&#39;" src="https://akphoto2.ask.fm/a5b/55ee9/e808/4b63/8799/c04c81425948/original/3310423.jpg" /></a></div>
</div>

  <div class="streamItemContent streamItemContent-footer">
    
    
# this is the pattern when there is only txt

  <div class="answerWrapper">
    <p class="streamItemContent streamItemContent-answer">Eyes????</p>
    <p class="readMore">
      <a class="button" data-action="ReadMore" href="#">View more</a>
    </p>
  </div>

    

  <div class="streamItemContent streamItemContent-footer">

In [ ]:
## finding metioning pattern 

<a dir="ltr" href="/kiddflava">@kiddflava</a>

<a dir="ltr" href="/sarmadnauman">@sarmadnauman</a>

<a dir="ltr" href="/rossy63">@rossy63</a> 

## it's the same in answer or question



## OK, what else? lets now clean the mentions

In [246]:
def getMention(txt):
    p = re.compile('<a dir="ltr" href="(.+?)</a>')
    new_txt = p.sub('$mention',txt)
    return new_txt
f = lambda x : getMention(x)

df['answer'] = df['answer'].apply(f)
df['question'] = df['question'].apply(f)

## Now let's clean the emotiocons 

In [249]:
df.answer_links.unique()

array(['',
       'https://akphoto2.ask.fm/ae6/acfd0/dce2/48c9/bbf4/1787d0f73892/large/381671.jpg',
       'https://akphoto2.ask.fm/05a/fdd68/d7e8/4312/b4bb/7aefd5435bce/large/797103.jpg',
       ...,
       'https://akphoto4.ask.fm/034/532/292/80003002-1r4an51-a6630t8kqg7d7rf/large/avatar.jpg',
       'https://akphoto2.ask.fm/644/55c29/b6bb/47be/b412/0b2340ec7aa2/large/206097.jpg',
       'https://akphoto2.ask.fm/285/215/621/710003000-1rk89j1-hbojjpla9opehf0/large/1560394_245162575673405_4325840948489570080_n.jpg'], dtype=object)